## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-06-58-48 +0000,nyt,Iran’s Leadership Stumbles After War and Unrest,https://www.nytimes.com/2026/01/24/world/middl...
1,2026-02-03-06-55-00 +0000,wsj,"Publicis Says Account Wins, AI Demand Lift Top...",https://www.wsj.com/business/earnings/publicis...
2,2026-02-03-06-55-00 +0000,wsj,"U.S. Treasury Yields Edge Higher, Steepening C...",https://www.wsj.com/finance/investing/jgb-futu...
3,2026-02-03-06-40-00 +0000,wsj,Akzo Nobel Paints Rosy Midterm Picture Despite...,https://www.wsj.com/business/earnings/akzo-nob...
4,2026-02-03-06-39-00 +0000,wsj,China Bans Retractable Door Handles on Cars as...,https://www.wsj.com/business/autos/china-bans-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2577/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,47
127,epstein,25
341,ice,17
47,deal,13
42,india,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,114
220,2026-02-02-18-39-33 +0000,nypost,"Trump says Jeffrey Epstein, Michael Wolff ‘con...",https://nypost.com/2026/02/02/us-news/trump-sa...,95
112,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...,93
347,2026-02-02-10-30-00 +0000,wsj,"Heard on the Street: Kevin Warsh, President Tr...",https://www.wsj.com/economy/central-banking/sa...,93
87,2026-02-03-00-37-12 +0000,nyt,"Trump Announces Initial Trade Deal With India,...",https://www.nytimes.com/2026/02/02/us/politics...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,114,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
49,81,2026-02-03-02-16-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
50,51,2026-02-03-02-12-34 +0000,nypost,Wild moment ICE agents chase down illegal migr...,https://nypost.com/2026/02/02/us-news/video-sh...
94,48,2026-02-03-00-07-20 +0000,nypost,Norway’s Epstein-tainted princess in yet anoth...,https://nypost.com/2026/02/02/world-news/norwa...
347,46,2026-02-02-10-30-00 +0000,wsj,"Heard on the Street: Kevin Warsh, President Tr...",https://www.wsj.com/economy/central-banking/sa...
112,46,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
153,43,2026-02-02-22-05-09 +0000,nypost,DHS chief Kristi Noem OKs body cameras for fed...,https://nypost.com/2026/02/02/us-news/dhs-chie...
121,42,2026-02-02-22-51-30 +0000,cbc,Arizona police suspect Today show host's mothe...,https://www.cbc.ca/news/world/nancy-guthrie-mi...
201,41,2026-02-02-19-36-47 +0000,cbc,Trump says he'll lower tariffs on India after ...,https://www.cbc.ca/news/world/trump-modi-russi...
183,39,2026-02-02-20-20-26 +0000,wapo,Rafah crossing between Gaza and Egypt reopens ...,https://www.washingtonpost.com/world/2026/02/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
